# logistic regression
binary classification (d size의 1D vector가 주어졌을 때 0 또는 1에 어느쪽에 가까운지를 판단)

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

y가 H(x)에 가까우면 cost도 0에 가깝고, 아니면 cost가 높음
- W의 차원은 d x 1
- X의 차원은 m x d (m개의 sample)
- W와 X를 곱한 후에 sigmoid 이용에 0과 1에 근사하게 함 (m x 1)

H(X)는 P(X=1)에 근사함. 
- $$H(X) = P(X=1;W) = 1 - P(X=0;W)$$

Gradient Descent로 weight update

$$ W := W - \alpha \frac{\partial}{\partial W} cost(W) $$

 - $\alpha$: Learning rate

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## Training Data

In [ ]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape) # 6 x 2
print(y_train.shape) # 6 x 1 

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

In [5]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [6]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [8]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor(1)))

# sigmoid 식은 torch에서 제공
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

print(hypothesis)
print(hypothesis.shape)

1/(1+e^{-1}) equals:  tensor([0.7311])
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

In [12]:
losses = -(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))
print(losses)

cost = losses.mean()
print(cost)

tensor([0.6931], grad_fn=<NegBackward0>)
tensor(0.6931, grad_fn=<MeanBackward0>)


In [14]:
cost = F.binary_cross_entropy(hypothesis, y_train)
print(cost)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)


## Whole Training Procedure

In [16]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {:4d}/{} Cost:{:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost:0.693147
Epoch  100/1000 Cost:0.134722
Epoch  200/1000 Cost:0.080643
Epoch  300/1000 Cost:0.057900
Epoch  400/1000 Cost:0.045300
Epoch  500/1000 Cost:0.037261
Epoch  600/1000 Cost:0.031672
Epoch  700/1000 Cost:0.027556
Epoch  800/1000 Cost:0.024394
Epoch  900/1000 Cost:0.021888
Epoch 1000/1000 Cost:0.019852


## Evaluation

In [ ]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])
print(y_train[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)
tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [20]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


## Higher Implementation with Class

In [31]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [33]:
model = BinaryClassifier()

## Higher Implementation with class

In [29]:
# data

x_train = torch.FloatTensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
y_train = torch.FloatTensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])

In [34]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 Cost: 0.760860 Accuracy 60.00%
Epoch   10/100 Cost: 0.281061 Accuracy 100.00%
Epoch   20/100 Cost: 0.176152 Accuracy 100.00%
Epoch   30/100 Cost: 0.128324 Accuracy 100.00%
Epoch   40/100 Cost: 0.100729 Accuracy 100.00%
Epoch   50/100 Cost: 0.082756 Accuracy 100.00%
Epoch   60/100 Cost: 0.070133 Accuracy 100.00%
Epoch   70/100 Cost: 0.060792 Accuracy 100.00%
Epoch   80/100 Cost: 0.053608 Accuracy 100.00%
Epoch   90/100 Cost: 0.047917 Accuracy 100.00%
Epoch  100/100 Cost: 0.043300 Accuracy 100.00%
